In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-174524
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-174524


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = 'EduardoCluster'
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)


In [3]:
from azureml.core import Environment

#sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path= './conda_dependencies.yml')
sklearn_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./',compute_target=cpu_cluster_name,script='train.py',environment=sklearn_env)

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C':choice(0.001,0.1,1,10,20),'--max_iter':choice(10,20,50,100,150)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


# Create a SKLearn estimator for use with train.py
est = src

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,policy=policy,hyperparameter_sampling=ps,primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=12)


In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)

In [11]:
import joblib
# Get your best run and save the model from that run.

best_model_acc = hyperdrive_run.get_best_run_by_primary_metric()

print("Best run metrics :",best_model_acc.get_metrics())

print("Best run details :",best_model_acc.get_details())



Best run metrics : {'Regularization Strength:': 1.0, 'Max iterations:': 100, 'Accuracy': 0.9108750632271118}
Best run details : {'runId': 'HD_25a92b05-d2c3-4d63-beef-1b1d89216bbf_10', 'target': 'EduardoCluster', 'status': 'Completed', 'startTimeUtc': '2022-01-11T15:34:37.510476Z', 'endTimeUtc': '2022-01-11T15:37:06.509299Z', 'services': {}, 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '589ea24b-5246-421f-8b4e-3f20b377c027', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatas

In [13]:
# we can get the filenames in order to register the best model
print(best_model_acc.get_file_names())

['logs/azureml/17_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']


In [14]:
model = best_model_acc.register_model(model_name='best-hyperdrive', model_path='outputs/model.joblib')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['logs/azureml/17_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/rslex.log', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['logs/azureml/17_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path ="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path)


In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
autoML_run = exp.submit(automl_config, show_output = False)
autoML_run.wait_for_completion()

2022-01-11:15:48:25,206 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-01-11:15:48:25,225 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-01-11:15:48:57,459 INFO     [utils.py:157] NumExpr defaulting to 2 threads.
2022-01-11:15:48:58,313 WARNING  [local_experiment_driver.py:148] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_8b438ae2-26d9-4360-9752-d2a14c5dbc07,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2022-01-11:16:15:55,713 INFO     [explanation_client.py:332] Using default datastore for uploads
2022-01-11:16:16:03,138 WARNING  [explanation_utils.py:92] Cannot serialize JSON possibly due to timestamp, retrying


{'runId': 'AutoML_8b438ae2-26d9-4360-9752-d2a14c5dbc07',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-01-11T15:49:04.221176Z',
 'endTimeUtc': '2022-01-11T16:14:08.74586Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-174524","workspace_name":"quick-starts-ws-174524","region":"southcentralus","compute_target":"local","spark_service":null

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###